<a href="https://colab.research.google.com/github/k-sakamaki/python_stan/blob/main/pystan_inst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import stan
import nest_asyncio
nest_asyncio.apply()

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}

"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Building...



Building: 39.3s, done.Messages from stanc:
Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 1.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.13 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.14 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!


In [ ]:
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,tau,eta.1,...,eta.7,eta.8,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6,theta.7,theta.8
draws,,,,,,,,,,,,,,,,,,,,,
0,-40.006857,0.600980,0.355501,3.0,7.0,0.0,44.840333,9.602765,1.700330,-0.674279,...,-0.657092,-0.851239,8.456268,6.433645,10.288361,9.335216,8.114231,10.768783,8.485492,8.155378
1,-38.360798,1.000000,0.298242,3.0,7.0,0.0,42.830817,4.694711,6.572202,-0.681739,...,-0.631193,0.956955,0.214185,8.079532,-3.309726,7.634535,1.377127,-0.357276,0.546382,10.984011
2,-36.888415,0.893117,0.293988,3.0,15.0,0.0,38.693508,10.375219,10.387525,1.273871,...,0.206319,-0.172160,23.607589,11.388530,17.365658,7.137600,-0.396582,-7.634052,12.518366,8.586902
3,-39.130649,0.965947,0.320275,4.0,15.0,0.0,46.352856,6.688797,5.591119,2.283668,...,1.411162,-0.456256,19.457058,0.737002,11.237984,9.232702,3.906940,7.705875,14.578770,4.137812
4,-37.275941,0.997449,0.355501,4.0,15.0,0.0,41.567184,7.285071,2.879834,0.743093,...,0.054176,-0.662775,9.425056,4.478288,8.152205,5.719658,8.068804,8.097466,7.441087,5.376388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-39.061618,0.995146,0.320275,3.0,7.0,0.0,49.553441,14.275365,9.812646,-0.080445,...,-1.112591,0.385024,13.485983,24.593734,9.565147,13.153894,12.396348,2.423207,3.357903,18.053469
3996,-41.435483,0.961733,0.355501,3.0,7.0,0.0,43.977505,12.089312,11.226074,1.750700,...,0.635751,0.271290,31.742805,14.669722,-4.410845,7.031521,25.283402,-2.983618,19.226294,15.134833
3997,-42.792561,0.987394,0.298242,4.0,15.0,0.0,46.589395,2.780286,0.098217,-0.016928,...,1.087653,-1.574422,2.778624,2.736104,2.797609,2.673697,2.825944,2.819155,2.887112,2.625651
